# Patch Generation : 
## FPS Modification : 

In [48]:
import open3d as o3d
import os
path = "../experiment/test/redandblack/predlift/r01/redandblack.ply.bin.decoded.ply"
import numpy as np
from pyntcloud import PyntCloud

def fps(points, n_samples):
    '''
    Description : This function takes as input the full Point cloud, and
    returns n_samples sampled points using the FPS algorithm
    '''
    # Represent the points by their indices in points
    points_left = np.arange(len(points)) # [P]

    # Initialise an array for the sampled indices
    sample_inds = np.zeros(n_samples, dtype='int') # [S]

    # Initialise distances to inf
    dists = np.ones_like(points_left) * float('inf') # [P]

    # Select a point from points by its index, save it
    selected = 0
    sample_inds[0] = points_left[selected]

    # Delete selected 
    points_left = np.delete(points_left, selected) # [P - 1]

    # Iteratively select points for a maximum of n_samples
    for i in range(1, n_samples):
        # Find the distance to the last added point in selected
        # and all the others
        last_added = sample_inds[i-1]
        
        dist_to_last_added_point = (
            (points[last_added][0:3] - points[points_left][:,0:3])**2).sum(-1) # [P - i]
        # If closer, updated distances
        dists[points_left] = np.minimum(dist_to_last_added_point, 
                                        dists[points_left]) # [P - i]

        # We want to pick the one that has the largest nearest neighbour
        # distance to the sampled points
        selected = np.argmax(dists[points_left])
        sample_inds[i] = points_left[selected]

        # Update points_left
        points_left = np.delete(points_left, selected)

    return points[sample_inds]

In [3]:
# Reading the point cloud using PCD
pcd = o3d.io.read_point_cloud("../experiment/test/redandblack/predlift/r01/redandblack.ply.bin.decoded.ply")
print(pcd)
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] Read PLY failed: unable to read file: ../experiment/test/redandblack/predlift/r01/redandblack.ply.bin.decoded.ply
PointCloud with 757691 points.


In [4]:
pcd_xyz = np.asarray(pcd.points)
pcd_rgb = np.asarray(pcd.colors)

In [5]:
full_pc = np.concatenate((pcd_xyz,pcd_rgb),axis=1)

In [50]:
points = fps(full_pc,1000)

In [54]:
pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(points[:,0:3])
pcd2.colors = o3d.utility.Vector3dVector(points[:,3:6])
o3d.io.write_point_cloud("sampled.ply", pcd2)

    # Load saved point cloud and visualize it
pcd_load = o3d.io.read_point_cloud("sampled.ply")
o3d.visualization.draw_geometries([pcd_load])

## Nearest Neighbours Search : 

In [130]:
def fps_nn(full_pc , sampled_pc , nn=2048):
    '''
    Description : This function takes as input the full Point cloud, a set of sampled
    points and the number of neighbours desired for each patch.
    It returns the nn indexes of nearest points to the sampled points
    '''
    patches = []
    for point in sampled_pc : # [ns , 6]
        # Calculate distances
        dists = ((point[0:3] - full_pc[:,0:3])**2).sum(-1)
        patch = sorted(enumerate(dists), key=lambda i: i[1])
        patches.append(patch[0:2048])
    return patches
patches_indexes = fps_nn(full_pc,points)

In [136]:
type(patches_indexes)

list

In [117]:
full_pc[0]

array([2.92000000e+02, 1.50000000e+01, 1.95000000e+02, 0.00000000e+00,
       1.84313725e-01, 1.33333333e-01])

In [154]:
import pickle

def write_list(a_list):
    # store list in binary file so 'wb' mode
    with open('listfile', 'wb') as fp:
        pickle.dump(names, fp)
        print('Done writing list into a binary file')

# Read list to memory
def read_list():
    # for reading also binary mode is important
    with open('listfile', 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

In [152]:
write_list(patches_indexes)

Done writing list into a binary file


In [ ]:
r_names = read_list()
print('List is', r_names)

In [167]:
patches = []
for patch_index_i in patches_indexes:
    small_patch =[]
    for index in patch_index_i :
        small_patch.append(full_pc[index[0]].tolist())
    patches.append(small_patch)

In [172]:
np_patches = np.asarray(patches)

In [186]:
pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(np_patches[700][:,0:3])
pcd2.colors = o3d.utility.Vector3dVector(np_patches[700][:,3:6])
o3d.io.write_point_cloud("exp_patch.ply", pcd2)
    # Load saved point cloud and visualize it
pcd_load = o3d.io.read_point_cloud("exp_patch.ply")
o3d.visualization.draw_geometries([pcd_load])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [180]:
for i in range(0,len(np_patches)):
    pc = o3d.geometry.PointCloud()
    pc.points= o3d.utility.Vector3dVector(np_patches[i][:,0:3])
    pc.colors= o3d.utility.Vector3dVector(np_patches[i][:,3:6])
    name= "red_black_patches/patch_"+str(i)+".ply"
    o3d.io.write_point_cloud(name, pc)